<a href="https://colab.research.google.com/github/fybx/python-egitimi/blob/main/VeriIsleme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFT & FTT Python Egitimi

TFT, Uludag Teknoloji ve Felsefe Toplulugu Instagram: https://instagram.com/uludagtft

FTT, Uludag Futuristik Tip Toplulugu Instagram: https://www.instagram.com/futuristiktip/

## Veri Isleme ornek projesi

Merhaba! Bu ornek projede, bir veri merkezinin ortam izleme ve kontrol sisteminin tuttugu gunluk verileri iceren bir metin belgesini okuyup Python'da isleyerek asagidaki hedefleri tamamlayacagiz:

1. **ortalama ortam sicakligi**ni ve bazi istatistiksel degerleri tespit edecek
2. veri noktalarinin zaman icerisindeki degisimini gosteren grafigi cikaracak
3. bu grafikten elde edilenler dogrultusunda veri uzerinde gelistirilebilecek farkli yaklasimlari arastiracak
4. saatlik ortalama sicakliklari bulmak icin yeni bir fonksiyon tasarlayacak
4. **saatlik ortalama sicakliklari**n grafigini cikaracak

### Baslangic

Baslamak icin oncelikle ihtiyacimiz olan kutuphaneleri Python calistirma ortamimiza dahil ediyoruz.

1. `datetime` modulunden `datetime` ve `timedelta` adli iki nesne
2. `matplotlib` modulunun `pyplot` nesnesini "plt" adiyla
3. ve `csv` modulunun icerigi


In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import csv

### Verinin Aktarimi

Elimizdeki buyuk ve karisik veri kaynagini inceleyebilmek ve islemek icin su an icinde oldugumuz ve komutlarla kontrol ettigimiz bu ortama bir sekilde aktarmaliyiz. Bunun icin `open(dosya_adi, izinler)` seklinde cagirdigimiz bir fonksiyondan yardim alacagiz.

In [ ]:
dosya_adi = 'fan_control.log'
with open(dosya_adi, 'r') as dosya:
  satirlar = [satir for satir in dosya.readlines() if satir.replace('\n', '')]
print('Satir sayisi: ', len(satirlar))
satirlar[0:4]

### Veriyi Cikartmak: On Hazirlik

Bu asamada elimizdeki devasa verinin gereksiz kisimlarindan kurtulmamiz, elimizdeki problemi cozmek icin yeterli olan kadarini ayristirmamiz gerekiyor.

Butun bunlari yaparken dogru iliskideki verileri aldigimiza dikkat etmeliyiz, eger parametrelerimizi dogru belirleyemezsek ilerleyen asamalarda sorun yasayabilir, **daha kotusu** sorun yasamadigimiz icin hatali bulgular elde ederek gerceklikten uzak bir sonuca varabiliriz.

In [ ]:
zaman_damgalari, sicakliklar = [], []
zaman_formati = '%Y-%m-%d %H:%M:%S'

In [ ]:
for satir in satirlar:
  if 'CPU Temperature' in satir:
    zaman_damgasi = satir.split('] ')[0][1:]
    sicaklik      = satir.split('CPU Temperature: ')[1][:-3]

    zaman = datetime.strptime(zaman_damgasi, zaman_formati)
    sicaklik = float(sicaklik[:-2])

    zaman_damgalari.append(zaman)
    sicakliklar.append(sicaklik)

In [ ]:
gecerli_veri_kirli_veri_orani = len(sicakliklar) / len(satirlar) * 100
print(f"%{gecerli_veri_kirli_veri_orani:.3f}")

### Veri Uzerinde Calismak: Hedef 1

Ortam izleme ve kontrol sisteminin calistigi sure boyunca biriktirdigi zaman ve sicaklik verisi uzerinden, bu genel veri setinin aritmetik ortalamasini bularak baslayalim.


#### Aritmetik Ortalama

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/66c77a2dae0af5783281635d538f0fde9b807589)

In [ ]:
aritmetik_ortalama = sum(sicakliklar) / len(sicakliklar)
print(f"{aritmetik_ortalama:.2f}")

#### Geometrik Ortalama

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/d8b67da21f4b58d3121ef21e0c5a9d040a6b65ce)

In [ ]:
def geometrik_ortalama(veri_noktalari):
  n = len(veri_noktalari)
  carpim = 1
  for eleman in veri_noktalari:
    carpim = carpim * eleman

  return carpim ** (1 / n)

In [ ]:
geometrik_ortalama(sicakliklar)

In [ ]:
def geometrik_ortalama2(veri_noktalari):
  n = len(veri_noktalari)
  carpim = 1
  for eleman in veri_noktalari:
    carpim = carpim * (eleman ** (1 / n))

  return carpim

In [ ]:
geometrik_ortalama2(sicakliklar)

### Veri Uzerinde Calismak: Gorsellestirme

Elimizdeki upuzun bu bir boyutlu listeyi farkli sekillerde resmederek verinin sahip oldugu trendleri gorebilir, sirada hangi islemin oldugu ve ne yapmamiz gerektigine dair daha iyi bir izlenim edinebiliriz.

Bunu bir labirentte dolasmak gibi (1-boyutlu) hayal edebilirsiniz. Bakis acinizi degistirebilseydiniz (kus bakisi baktiginizi dusunun, boylece 2-boyutlu bir veriyle ugrasmis olacaksiniz) labirenti cozmek inanilmaz hizli bir sekilde kisalirdi.

In [ ]:
plt.figure(figsize=(24, 6))
plt.scatter(zaman_damgalari, sicakliklar, marker='o')
plt.xlabel('Tarih')
plt.ylabel('CPU Sicakligi (°C)')
plt.title('Zaman icerisinde CPU Sicaklik degisimi')
plt.grid(True)
plt.show()

Korkunc bir goruntu ile karsi karsiyayiz, gorsellestirme her ne kadar yardimci olsa bile boyle bir goruntuden mana cikarabilmek oldukca zor.

Ilk bakista bizi neden bu kadar yogun ve ust uste binmis bir grafigin karsiladigini anlamak guc olabilir ancak hizlica `satirlar` degiskeninde saklanan verilere goz atarsaniz (`satirlar[aralik_basi:aralik_sonu]`) ortam kontrol sisteminin sicaklik olcumlerini dakika basina yaptigini goreceksiniz.

Dakika basina yapilan degisimler bu devasa veri seti icerisinde **yalnizca bir gunu** incelemek isteseydik faydali olabilirdi. Bizim hedefimiz ise gecen zaman icerisindeki genel trende dair bir ipucu edinebilmek.

Oyleyse simdi, saatlik ortalamalari hesaplayan bir yontem gelistirerek grafigi iyilestirelim.

### Veri Uzerinde Calismak: Anlam Arayisi

Alacagi iki parametre (`zaman_damgalari` ve `sicakliklar` listeleri) uzerinden saatlik ortalamalari hesaplayacak bir fonksiyon tasarlayalim:

Fonksiyonu tasarlamanin birinci ve en kritik asamasi algoritmanin netlestirilmesidir. Algoritmayi tasarlarken ise programlama dilini kullanmak gecerli bir tercih olabilse de kolaylik saglamasi acisindan Turkce bir sekilde ifade edebiliriz.

#### Algoritma tasarimi

Her bir veri noktasini, gun degerinden bagimsiz olarak SS:DD formunda dusunelim. Bir dongu yardimiyla elimizdeki her veri noktasinin, ondan bir sonra gelen saatten kucuk olup olmadigina bakarak hangi saat hanesine dustugune karar verebiliriz. Boylece efektif bicimde veri noktalarini yer aldiklari saat dilimlerine ayirabiliriz.

Bu yaptigimiz aslinda nerede durmamiz gerektigini kontrol etmekten ibarettir. Eger durmamiz gereken noktaya vardiysak (toplamaya basladigimiz saatten bir sonrakine gecildiyse) simdiye dek topladigimiz degerleri, kac tane olduklarina bolerek o saatin ortalamasini alacagiz.

#### Algoritmanin sozde-kodu

```
fonksiyon saatlik_ortalama (zaman_listesi, sicaklik_listesi) seklinde iki parametre alir

    saatlik_ortalamalar = bos liste tanimla

    su_an_saat = zaman_listesi'nde yer alan ilk saat  olarak ayarla
    toplam_sicaklik = 0.0
    toplam_nokta = 0

    sicaklik_listesindeki her k. (k'inci) nokta icin:
        eger k.inci nokta bir sonraki saatten kucukse:
            toplam_sicaklik += k. sicaklik
            toplam_nokta += 1
        degilse
            eger bu saat diliminde hic nokta yoksa
                saatlik_ortalamalar listesine bu saatin bos oldugu ibaresini ekle (su_an_saat, None)
            degilse
                ortalama = toplam_sicaklik / toplam_nokta
                saatlik_ortalamalar listesine saat ve ortalama bilgisini ekle (su_an_saat, ortalama)
        
        # siradaki saate gecilecek, degiskenleri hazirla
        su_an_saat += 1
        toplam_sicaklik = 0.0
        toplam_nokta = 0
    
    # bu asamada tek bir eksik kalmakta:
    # dongu icindeki tespit mekanizmamiz son saatten hemen oncesine kadar olan noktalari isleyebildi
    # son saati de yukaridaki sekilde islemeliyiz

    ortalama = toplam_sicaklik / toplam_nokta
    saatlik_ortalamalar listesine saat ve ortalama bilgisini ekle (su_an_saat, ortalama)

    saatlik_ortalamalar dondur
```

In [ ]:
def saatlik_ortalamayi_bul(zaman_listesi, sicaklik_listesi):
  saatlik_ortalamalar = []
  su_an_saat = zaman_listesi[0].replace(minute=0, second=0)

  toplam_sicaklik = 0.0
  toplam_nokta = 0

  for k in range(len(sicaklik_listesi)):
    if zaman_listesi[k] < su_an_saat + timedelta(hours=1):
      toplam_sicaklik += sicaklik_listesi[k]
      toplam_nokta +=1
    else:
      if toplam_nokta > 0:
        ortalama = toplam_sicaklik / toplam_nokta
        saatlik_ortalamalar.append((su_an_saat, ortalama))
      else:
        saatlik_ortalamalar.append((su_an_saat, None))

      su_an_saat += timedelta(hours=1)
      toplam_sicaklik = 0
      toplam_nokta = 0

  ortalama = toplam_sicaklik / toplam_nokta
  saatlik_ortalamalar.append((su_an_saat, ortalama))
  return saatlik_ortalamalar


In [ ]:
ortalamalar = saatlik_ortalamayi_bul(zaman_damgalari, sicakliklar)
ortalamalar[0:12]

### Veri ve Isleme: Final

In [ ]:
nokta_saat, nokta_sicaklik = [], []
for nokta in ortalamalar:
  nokta_saat.append(nokta[0])
  nokta_sicaklik.append(nokta[1])

In [ ]:
plt.figure(figsize=(24,6))
plt.scatter(nokta_saat, nokta_sicaklik, marker='.')
plt.xlabel('Tarih')
plt.ylabel('CPU Sicakligi (°C)')
plt.title('Zaman icerisinde CPU Sicaklik degisimi')
plt.grid(True)
plt.show()

## Kapanis

Ferit Yiğit BALABAN, <[fyb@fybx.dev][llmail]>

[My Website][llwebsite] • [My Bento][llbento] • [X][llx] • [LinkedIn][lllinkedin]

2024

[llmail]: mailto:fyb@fybx.dev
[llwebsite]: https://fybx.dev
[llbento]: https://bento.me/balaban
[llx]: https://x.com/fybalaban
[lllinkedin]: https://linkedin.com/in/fybx